In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Part 3
</center>
</h1>
<div class=h1_cell>
<p>
........................
</div>

In [2]:
import pandas as pd
import os

home_path = os.path.expanduser('~')

file_path = '/Documents/CIS 399/'

file_name = 'wrangled_game_table_trees.csv'

game_table = pd.read_csv(home_path + file_path + file_name)
pd.__version__

u'0.20.3'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
os.chdir(home_path + '/Documents/CIS 399/funcs/datascience_1')
!git pull

Already up to date.


In [5]:
import sys
sys.path.append(home_path + '/Documents/CIS 399/funcs/datascience_1')
from week7 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 vote_taker	 



In [6]:
game_table.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan,gls_binned,gls_low,gls_average,gls_high,gls_nan,forest1_oob,forest1_oob_types,forest2_oob,forest2_oob_types,forest3_oob,forest3_oob_types
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.000000,76.00,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0,high,0,0,1,0,1,true_positive,1,true_positive,1,true_positive
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2008.000000,82.00,1,average,0,1,0,0,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,2009.000000,80.00,1,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0,0,false_negative,1,true_positive,1,true_positive
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1,low,1,0,0,0,low,1,0,0,0,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive


<h1>
<center>
Tuning the Random Forest Model
</center>
</h1>
<div class=h1_cell>
<p>
In this .........
<p>
We will start by looking back at our best random forest from Part 2.
</div>

In [7]:
forest2_oob_types = game_table['forest2_oob_types'].value_counts()

print((accuracy(forest2_oob_types), f1(forest2_oob_types), informedness(forest2_oob_types)))

(0.75327471738740359, 0.012921751615218951, 0.005957167824620635)


<hr>
<h2>
Analyzing the Forest
</h2>
<p>
<div class=h1_cell>
<p>
We're going to analyze the forests individually and try to improve it.
<p>
This next function takes a forest and compiles a matrix of all of the tree votes for the forest.
</div>

In [25]:
def all_votes(forest, table):
    result = []
    for i in range(len(table)):
        row_preds = []
        for x in range(len(forest)):
            row = table.loc[i]
            pred = tree_predictor(row, forest[x])
            row_preds.append(pred)
        result.append(row_preds)
    return result

In [26]:
forest_columns = [
    #North American Sales
    'nas_low', 'nas_average', 'nas_high',
    #European Sales
    'eus_low', 'eus_average', 'eus_high',
    #Japanese Sales
    'jps_low', 'jps_average', 'jps_high',
    #Other Sales
    'ots_low', 'ots_average', 'ots_high',
    #Global Sales
    'gls_low', 'gls_average', 'gls_high'
]


# I wrote this function for oob testing purposes
def oob_testing(forest, table):
    new_table = table.loc[forest[0]['oob']]
    
    for i in range(1, len(forest)):
        temp = table.loc[forest[i]['oob']]
        new_table = new_table.append(temp)
    return new_table

In [27]:
#Test on our forest
init_forest = forest_builder(game_table, forest_columns, 'rating_over_70', hypers={'total-trees':10})

oob_table = oob_testing(init_forest, game_table)
best_forest = forest_builder(oob_table, forest_columns, 'rating_over_70', hypers={'total-trees':10})

In [28]:
votes = all_votes(best_forest, game_table)

In [29]:
votes

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 1, 0, 1, 0, 1, 0, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0,

<div class=h1_cell>
<p>
Find the number of rows with unanimous predictions.
</div>

In [30]:
def unanimous_rows(all_votes):
    unan_list = []
    for vote in all_votes:
        unan = True
        curr = vote[0]
        for i in vote:
            if i != curr:
                unan = False
                break
        if(unan):
            unan_list.append(vote)
    return unan_list

In [31]:
unan_rows = unanimous_rows(votes)

In [32]:
len(unan_rows)

16683

<div class=h1_cell>
<p>
There is a substantial number of unanimous rows in this forest.
</div>

<div class=h1_cell>
<p>
Now we're going to find which trees always produce the same prediction.
</div>

In [21]:
def never_change(all_votes):
    constant_trees = []
    for i in range(len(all_votes[0])):
        same = True
        curr = all_votes[0][i]
        for j in range(len(all_votes)):
            if all_votes[j][i] != curr:
                same = False
                break
        if(same):
            constant_trees.append(i)
    return constant_trees

In [33]:
never = never_change(votes)

In [34]:
len(never)

0

<h1>
<center>
Non-empty 'Critic_Score'
</center>
</h1>
<div class=h1_cell>
<p>
Going to create a sub table of rows which have values for 'Critic_Score'
<p>
This sub table is going to help us get the mean for 'Critic_Score'
</div>

In [22]:
cs_table = game_table.loc[game_table['Critic_Score'].notnull()]
print(len(cs_table))

8137


<h1>
<center>
Label Centroids
</center>
</h1>
<div class=h1_cell>
<p>
This algorithm I wrote goes through every row in our non-empty table in order to come up with a mean for each of our centroids.
<p>
This mean is stored in 'mean_label' in the centroid.
</div>

In [23]:
def compute_centroid_labels(centroids, focus_table, focus_column, features, k):
    
    for i in range(len(centroids)):
        centroids[i]['sum']= []
        centroids[i]['mean_label'] = 0
    for index, row in focus_table.iterrows():
        row_list = row_to_vect(row, features)
        (close_index, dist) = closest_centroid(centroids, row_list, k)
        centroids[close_index]['sum'].append(row[focus_column])
        
    for cent_index in range(len(centroids)):
        mean = sum(centroids[cent_index]['sum']) / len(centroids[cent_index]['sum'])
        centroids[cent_index]['mean_label'] = (mean)
    return centroids

In [24]:
centroids = compute_centroid_labels(final_centroids, cs_table, 'Critic_Score', features_used, k)

In [25]:
for i in range(k):
    print(centroids[i]['mean_label'])

86.55
71.1192833569
65.8158042745
72.1542898342
70.440464666


<h1>
<center>
Modify our Original Table
</center>
</h1>
<div class=h1_cell>
<p>
I wrote this algorithm to fill in empties in a table based on their closest centroid.
<p>
Only rows with an empty value for 'focus_column' will be affected.
</div>

In [26]:
def kmeans_fill(centroids, full_table, features, focus_column, k, col_name):
    
    full_table[col_name] = ""
    for item, value in full_table[focus_column].iteritems():
        if(pd.isnull(value)):
            row_list = row_to_vect(full_table.loc[item], features)
            (close_index, distance) = closest_centroid(centroids, row_list, k)
            full_table.loc[item, col_name] = centroids[close_index]['mean_label']
        else:
            full_table.loc[item, col_name] = value
    
    return full_table

In [27]:
game_table = kmeans_fill(centroids, game_table, features_used, focus_column, k, 'kmeans_Critic_Score')
game_table.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.000000,76
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2008.000000,82
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,2009.000000,80
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55


<h1>
<center>
Create our Binary Label
</center>
</h1>
<div class=h1_cell>
<p>
Going to create a binary colum for rows with a score over 70.
<p>
I'm going to base this off of our new kmeans column.
</div>

In [30]:
game_table['rating_over_70'] = game_table.apply(lambda row: 1 if row.kmeans_Critic_Score > 70 else 0, axis=1)

In [34]:
game_table.head(-1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.000000,76,1
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2008.000000,82,1
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,2009.000000,80,1
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1
5,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E,2006.000000,89,1
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E,2006.000000,58,0
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E,2009.000000,87,1
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1


<h1>
<center>
OHE
</center>
</h1>
<div class=h1_cell>
<p>
We're going to one hot encode some more binary columns based on the sales data. These columns are going to help us with future testing. 
</div>

In [6]:
bins = 3
bin_names = ['low', 'average', 'high']
game_table['nas_binned'] = pd.cut(game_table['NA_Sales'], bins, labels=bin_names)

In [7]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high


In [8]:
ohe_nas = pd.get_dummies(game_table['nas_binned'], prefix='nas', dummy_na=True)
game_table = game_table.join(ohe_nas)

In [9]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,...,Nintendo,E,2006.0,76.0,1,high,0,0,1,0


<div class=h1_cell>
<p>
We're going to continue doing this for the other sales columns in the table.
<p>
EU Sales

In [10]:
game_table['eus_binned'] = pd.cut(game_table['EU_Sales'], bins, labels=bin_names)

In [11]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,...,E,2006.0,76.0,1,high,0,0,1,0,high


In [12]:
ohe_eus = pd.get_dummies(game_table['eus_binned'], prefix='eus', dummy_na=True)
game_table = game_table.join(ohe_eus)

In [13]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,...,high,0,0,1,0,high,0,0,1,0


<div class=h1_cell>
<p>
JP Sales

In [ ]:
game_table['jps_binned'] = pd.cut(game_table['JP_Sales'], bins, labels=bin_names)

In [18]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high,0,0,1,0,high,0,0,1,0,average


In [19]:
ohe_jps = pd.get_dummies(game_table['jps_binned'], prefix='jps', dummy_na=True)
game_table = game_table.join(ohe_jps)

In [20]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0


<div class=h1_cell>
<p>
Other Sales

In [21]:
game_table['ots_binned'] = pd.cut(game_table['Other_Sales'], bins, labels=bin_names)

In [22]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high


In [23]:
ohe_ots = pd.get_dummies(game_table['ots_binned'], prefix='ots', dummy_na=True)
game_table = game_table.join(ohe_ots)

In [24]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0


<div class=h1_cell>
<p>
Global Sales

In [25]:
game_table['gls_binned'] = pd.cut(game_table['Global_Sales'], bins, labels=bin_names)

In [26]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan,gls_binned
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0,high


In [27]:
ohe_gls = pd.get_dummies(game_table['gls_binned'], prefix='gls', dummy_na=True)
game_table = game_table.join(ohe_gls)

In [28]:
game_table.head(1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan,gls_binned,gls_low,gls_average,gls_high,gls_nan
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.0,76.0,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0,high,0,0,1,0


<h1>
<center>
Part 1 Done
</center>
</h1>
<div class=h1_cell>
<p>
We now have the relevant empties filled with simple mean and kmeans values. Additionally, we now have our binary column to predict on in the next section. We also have our sales data one hot encoded into binary columns as well.
</div>

In [29]:
import os

home_path =  os.path.expanduser('~')

file_path = '/Documents/CIS 399/'

file_name = 'wrangled_game_table.csv'

game_table.to_csv(home_path + file_path + file_name, index=False)